# HW08: API

# 1. Choose an API
## a) Choose an API and briefly describe the type of data you can obtain from it.
For this assignment, I will be using openweathermap. I can collect and process weather data from different sources such as global and local weather models, satellites, radars and a vast network of weather stations. Data is available in JSON, XML, or HTML format.



## b) Provide a link to the API documentation
This is the link to the API documentation: https://openweathermap.org/api

## c) the base URL of the API you intend to use
This is the base URL of the API:https://openweathermap.org/forecast5


# 2. Authentication
## a) Briefly explain how the API authenticates the user
Just sign up with email and start using minute forecasts, hourly forecasts, history and other weather data in your applications for free. After signing up, users will get a specific email which is used to verify identity. After the verification, the exclusive API Key can be accessed via email to authenticate the user.

## b) Apply for an API key if necessary and provide the information (with relevant URL) how that can be done. Do not include the API key in the assignment submission.
The quick guide about API key application: https://openweathermap.org/appid

# 3. Send a Simple GET request
## a) Execute a simple GET request to obtain a small amount of data from the API


In [1]:
import requests
import os
import json

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path = '/Users/apple/Documents/GitHub/Zhenwei_Ma/hw08/hw08.env')
private_key = os.getenv('MY_ENV_VAR')

In [3]:
params={'appid':private_key,'q':'Los Angeles'}
r = requests.get('https://api.openweathermap.org/data/2.5/forecast',params=params)

In [4]:
print(json.dumps(r.json(), indent=2,sort_keys= True))

{
  "cod": 401,
  "message": "Invalid API key. Please see http://openweathermap.org/faq#error401 for more info."
}


## b) Check (and show) the status of the request

In [231]:
r.status_code

200

## c) Check (and show) the type of the response (e.g. XML, JSON, csv)

In [232]:
r.headers['content-type']

'application/json; charset=utf-8'

# 4. Parse the response and Create a dataset
## a) Take the response returned by the API and turn it into a useful Python object (e.g. a list, vector, or pandas data frame). Show the code how this is done.

In [237]:
import pandas as pd
weather_LA = r.json()
print(type(weather_LA))  # dictionary
print(weather_LA.keys()) # keys


<class 'dict'>
dict_keys(['cod', 'message', 'cnt', 'list', 'city'])


In [243]:
weather_LA_df = pd.DataFrame(weather_LA['list'])


In [244]:
weather_LA_df.head()

,dt,main,weather,clouds,wind,visibility,pop,sys,dt_txt
0,1637161200,"{'temp': 286.16, 'feels_like': 285.74, 'temp_m...","[{'id': 800, 'main': 'Clear', 'description': '...",{'all': 1},"{'speed': 0.36, 'deg': 185, 'gust': 0.64}",10000,0,{'pod': 'd'},2021-11-17 15:00:00
1,1637172000,"{'temp': 288.12, 'feels_like': 287.63, 'temp_m...","[{'id': 801, 'main': 'Clouds', 'description': ...",{'all': 19},"{'speed': 1.3, 'deg': 195, 'gust': 0.85}",10000,0,{'pod': 'd'},2021-11-17 18:00:00
2,1637182800,"{'temp': 291.95, 'feels_like': 291.4, 'temp_mi...","[{'id': 801, 'main': 'Clouds', 'description': ...",{'all': 22},"{'speed': 2.5, 'deg': 230, 'gust': 1.57}",10000,0,{'pod': 'd'},2021-11-17 21:00:00
3,1637193600,"{'temp': 292.96, 'feels_like': 292.41, 'temp_m...","[{'id': 801, 'main': 'Clouds', 'description': ...",{'all': 19},"{'speed': 3.03, 'deg': 242, 'gust': 2.4}",10000,0,{'pod': 'd'},2021-11-18 00:00:00
4,1637204400,"{'temp': 291.07, 'feels_like': 290.48, 'temp_m...","[{'id': 802, 'main': 'Clouds', 'description': ...",{'all': 46},"{'speed': 1.63, 'deg': 244, 'gust': 1.83}",10000,0,{'pod': 'n'},2021-11-18 03:00:00


In [246]:
print(json.dumps(weather_LA['list'][0]['wind'], indent=4, sort_keys=True))

{
    "deg": 185,
    "gust": 0.64,
    "speed": 0.36
}


In [248]:
sample_df = pd.DataFrame(weather_LA['list'][0]['wind'], index = [1])
sample_df.head()

,speed,deg,gust
1,0.36,185,0.64


## b) Using the API, create a dataset (in data frame format) for multiple records. I'd say a sample size greater than 100 is sufficient for the example but feel free to get more data if you feel ambitious and the API allows you to do that fairly easily. The dataset can include only a small subset of the returned data. Just choose some interesting features. There is no need to be inclusive here.

In [250]:
for col in weather_LA_df.columns:
   print(col)

dt
main
weather
clouds
wind
visibility
pop
sys
dt_txt


In [5]:
df = pd.DataFrame()
s = 0
for s in range(len(weather_LA['list'])):
    wind = pd.DataFrame(weather_LA['list'][s]['wind'], index = [s])
    df = pd.concat([df, wind])
    s = s + 1

df2 = pd.DataFrame()
t = 0
for t in range(len(weather_LA['list'])):
    time = pd.Series(weather_LA['list'][t]['dt_txt'], index = [t])
    time = pd.DataFrame(time)
    df2 = pd.concat([df2, time])
    t = t + 1
df2.columns = ['Time']

df = pd.concat([df, df2], axis=1)
df.head()  

NameError: name 'pd' is not defined

## c) Provide some summary statistics of the data. Include the data frame in a .csv file called data.csv with your submission for the grader.

In [255]:
df.describe()

,speed,deg,gust
count,40.000000,40.000000,40.000000
mean,1.414750,166.125000,1.516250
std,0.874044,93.477047,0.759604
min,0.170000,1.000000,0.400000
25%,0.647500,100.250000,0.900000
50%,1.210000,216.500000,1.305000
75%,2.250000,232.250000,2.102500
max,3.030000,298.000000,3.430000


In [256]:
df.to_csv('data.csv')

# 5. API Client
## a) API client function

In [258]:
def API_Client(appid, city = 'Los Angeles'):
    """
    The function can help collect 5 day / 3 hour forecast weather data for a specified city.
    
    Parameters
    ----------
    appid: str
        The API key which can be acquired by signing up on the website.
    city: str
        The specific city for which you want to predict weather data.
        
    Returns
    -------
    status: int
        The status codes show the HTTP status, 1XX means Informational, 2XX means Success which is what we want, 3XX means Redirection, 4XX means Client Error, 5XX means Server Error
    df: DataFrame
        The dataframe shows wind data 5 day / 3 hour for a specific city
    
    Examples
    --------
    >>> import API_Client
    >>> a = wea_api_key
    >>> b = 'Tokyo'
    >>> API_Client(a, b)
    (200,
     speed  deg  gust                 Time
 0    0.36  185  0.64  2021-11-17 15:00:00
 1    1.30  195  0.85  2021-11-17 18:00:00
 2    2.50  230  1.57  2021-11-17 21:00:00
 3    3.03  242  2.40  2021-11-18 00:00:00
 4    1.63  244  1.83  2021-11-18 03:00:00
 5    0.62  223  1.13  2021-11-18 06:00:00
 6    1.65  256  2.21  2021-11-18 09:00:00
 7    0.17  170  0.80  2021-11-18 12:00:00
 8    0.60  233  1.03  2021-11-18 15:00:00
 9    1.59  214  1.05  2021-11-18 18:00:00
 10   2.31  228  1.48  2021-11-18 21:00:00
 11   2.56  241  2.11  2021-11-19 00:00:00
 12   1.19  223  1.61  2021-11-19 03:00:00
 13   0.93  232  1.24  2021-11-19 06:00:00
 14   0.38  241  0.77  2021-11-19 09:00:00
 15   0.52  219  0.81  2021-11-19 12:00:00
 16   0.35  229  0.79  2021-11-19 15:00:00
 17   1.07  179  0.90  2021-11-19 18:00:00
 18   1.65  234  1.05  2021-11-19 21:00:00
 19   2.23  239  1.98  2021-11-20 00:00:00
 20   0.96  229  1.35  2021-11-20 03:00:00
 21   0.65  170  0.95  2021-11-20 06:00:00
 22   0.64  110  0.72  2021-11-20 09:00:00
 23   0.36  109  0.52  2021-11-20 12:00:00
 24   0.17   74  0.40  2021-11-20 15:00:00
 25   1.23  214  0.96  2021-11-20 18:00:00
 26   2.15  223  1.24  2021-11-20 21:00:00
 27   2.34  229  1.28  2021-11-21 00:00:00
 28   1.35  265  2.22  2021-11-21 03:00:00
 29   0.78   11  0.90  2021-11-21 06:00:00
 30   1.07    1  1.33  2021-11-21 09:00:00
 31   2.09   14  2.16  2021-11-21 12:00:00
 32   2.45   19  2.50  2021-11-21 15:00:00
 33   1.17   23  2.10  2021-11-21 18:00:00
 34   0.67  146  1.86  2021-11-21 21:00:00
 35   0.88  298  1.80  2021-11-22 00:00:00
 36   3.01   19  3.43  2021-11-22 03:00:00
 37   2.77   12  3.19  2021-11-22 06:00:00
 38   2.67   18  2.75  2021-11-22 09:00:00
 39   2.54    4  2.74  2021-11-22 12:00:00)

    """
    params = {'appid':appid, 'q': city}
    r = requests.get('https://api.openweathermap.org/data/2.5/forecast', params = params)
    status = r.status_code
    
    df = pd.DataFrame()
    s = 0
    for s in range(len(weather_LA['list'])):
        wind = pd.DataFrame(weather_LA['list'][s]['wind'], index = [s])
        df = pd.concat([df, wind])
        s = s + 1

    df2 = pd.DataFrame()
    t = 0
    for t in range(len(weather_LA['list'])):
        time = pd.Series(weather_LA['list'][t]['dt_txt'], index = [t])
        time = pd.DataFrame(time)
        df2 = pd.concat([df2, time])
        t = t + 1
    df2.columns = ['Time']

    df = pd.concat([df, df2], axis=1)
    
    return status, df

In [260]:
API_Client(wea_api_key, city = 'Tokyo')

(200,
     speed  deg  gust                 Time
 0    0.36  185  0.64  2021-11-17 15:00:00
 1    1.30  195  0.85  2021-11-17 18:00:00
 2    2.50  230  1.57  2021-11-17 21:00:00
 3    3.03  242  2.40  2021-11-18 00:00:00
 4    1.63  244  1.83  2021-11-18 03:00:00
 5    0.62  223  1.13  2021-11-18 06:00:00
 6    1.65  256  2.21  2021-11-18 09:00:00
 7    0.17  170  0.80  2021-11-18 12:00:00
 8    0.60  233  1.03  2021-11-18 15:00:00
 9    1.59  214  1.05  2021-11-18 18:00:00
 10   2.31  228  1.48  2021-11-18 21:00:00
 11   2.56  241  2.11  2021-11-19 00:00:00
 12   1.19  223  1.61  2021-11-19 03:00:00
 13   0.93  232  1.24  2021-11-19 06:00:00
 14   0.38  241  0.77  2021-11-19 09:00:00
 15   0.52  219  0.81  2021-11-19 12:00:00
 16   0.35  229  0.79  2021-11-19 15:00:00
 17   1.07  179  0.90  2021-11-19 18:00:00
 18   1.65  234  1.05  2021-11-19 21:00:00
 19   2.23  239  1.98  2021-11-20 00:00:00
 20   0.96  229  1.35  2021-11-20 03:00:00
 21   0.65  170  0.95  2021-11-20 06:00:00
 22  